## About Keras layers

All Keras layers have a number of methods in common:



In [1]:
from keras.layers import Dense, Input

inputs = Input((8,))
layer = Dense(8)

layer.get_weights()

Using TensorFlow backend.


[]

In [2]:
x = layer(inputs)

In [3]:
layer.name

'dense_1'

In [4]:
layer.__class__.__name__

'Dense'

In [5]:
layer.trainable = True

In [6]:
layer.get_weights()

[array([[-0.37464097, -0.27893341,  0.49695486,  0.4449653 ,  0.36643338,
         -0.52931106, -0.46890604, -0.45965397],
        [-0.40730417, -0.18583602,  0.37762654,  0.11463147,  0.28971136,
          0.57164758, -0.15994304,  0.32945877],
        [ 0.36391342,  0.14205736, -0.10808742, -0.4931342 ,  0.0342685 ,
          0.14510453, -0.23283812, -0.33516946],
        [-0.287334  , -0.13771147,  0.39885491, -0.13430265, -0.30147257,
         -0.46581769, -0.27736709,  0.57230049],
        [ 0.00393558,  0.27578139,  0.13987607,  0.35666347, -0.46667796,
         -0.05191928,  0.52654058,  0.44505733],
        [ 0.35879415, -0.42859054, -0.35855487,  0.17172635,  0.28866434,
          0.19636095, -0.11876026, -0.41038626],
        [-0.30232173,  0.24163556, -0.01312065, -0.18224221,  0.06398374,
          0.59276479, -0.04110891,  0.38710332],
        [ 0.38055694, -0.18147221,  0.1881265 ,  0.27507681,  0.35192412,
         -0.27093357,  0.18050861,  0.35254461]], dtype=float32),

In [7]:
import numpy as np

new_bias = np.array([ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.])
layer.set_weights([layer.get_weights()[0], new_bias])

In [8]:
layer.get_weights()

[array([[-0.37464097, -0.27893341,  0.49695486,  0.4449653 ,  0.36643338,
         -0.52931106, -0.46890604, -0.45965397],
        [-0.40730417, -0.18583602,  0.37762654,  0.11463147,  0.28971136,
          0.57164758, -0.15994304,  0.32945877],
        [ 0.36391342,  0.14205736, -0.10808742, -0.4931342 ,  0.0342685 ,
          0.14510453, -0.23283812, -0.33516946],
        [-0.287334  , -0.13771147,  0.39885491, -0.13430265, -0.30147257,
         -0.46581769, -0.27736709,  0.57230049],
        [ 0.00393558,  0.27578139,  0.13987607,  0.35666347, -0.46667796,
         -0.05191928,  0.52654058,  0.44505733],
        [ 0.35879415, -0.42859054, -0.35855487,  0.17172635,  0.28866434,
          0.19636095, -0.11876026, -0.41038626],
        [-0.30232173,  0.24163556, -0.01312065, -0.18224221,  0.06398374,
          0.59276479, -0.04110891,  0.38710332],
        [ 0.38055694, -0.18147221,  0.1881265 ,  0.27507681,  0.35192412,
         -0.27093357,  0.18050861,  0.35254461]], dtype=float32),

In [9]:
layer.input, layer.output, layer.input_shape, layer.output_shape

(<tf.Tensor 'input_1:0' shape=(?, 8) dtype=float32>,
 <tf.Tensor 'dense_1/BiasAdd:0' shape=(?, 8) dtype=float32>,
 (None, 8),
 (None, 8))

In [10]:
x = layer(x)

In [11]:
layer.input, layer.output, layer.input_shape, layer.output_shape

AttributeError: Layer dense_1 has multiple inbound nodes, hence the notion of "layer input" is ill-defined. Use `get_input_at(node_index)` instead.

In [12]:
layer.get_input_at(0), layer.get_output_at(0), layer.get_input_shape_at(0), layer.get_output_shape_at(0)

(<tf.Tensor 'input_1:0' shape=(?, 8) dtype=float32>,
 <tf.Tensor 'dense_1/BiasAdd:0' shape=(?, 8) dtype=float32>,
 (None, 8),
 (None, 8))

## Saving and Loading Individual Layer Configs

In [14]:
layer = Dense(32)
config = layer.get_config()
reconstructed_layer = Dense.from_config(config)
config

{'activation': 'linear',
 'activity_regularizer': None,
 'bias_constraint': None,
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'bias_regularizer': None,
 'kernel_constraint': None,
 'kernel_initializer': {'class_name': 'VarianceScaling',
  'config': {'distribution': 'uniform',
   'mode': 'fan_avg',
   'scale': 1.0,
   'seed': None}},
 'kernel_regularizer': None,
 'name': 'dense_3',
 'trainable': True,
 'units': 32,
 'use_bias': True}

In [15]:
from keras import layers

config = layer.get_config()
layer = layers.deserialize({'class_name': layer.__class__.__name__,
                            'config': config})

## Lambda Method

The lambda method allows you to make any stateless transformation (available in our backend which is tensorflow) to our tensors. Including transformation logic outside a layer will mess up any model, so make sure to put it inside a lambda layer.

The output shape is not necessary in tensorflow as it will try to impute it. That being said, for some complex functions it might be better to specify it yourself.

In [16]:
from keras.layers import Lambda
from keras import backend as K

Lambda(lambda x: x ** 2)

In [17]:
def antirectifier(x):
    x -= K.mean(x, axis=1, keepdims=True)
    x = K.l2_normalize(x, axis=1)
    pos = K.relu(x)
    neg = K.relu(-x)
    return K.concatenate([pos, neg], axis=1)

def antirectifier_output_shape(input_shape):
    shape = list(input_shape)
    assert len(shape) == 2  # only valid for 2D tensors
    shape[-1] *= 2
    return tuple(shape)

Lambda(antirectifier, output_shape=antirectifier_output_shape)

In [18]:
import tensorflow as tf
Lambda(tf.reduce_mean, output_shape=(1,))

In [19]:
def to_pow(x, po=2):
    return x ** po

Lambda(to_pow, arguments={'po':5})

## Writing your own Keras layers

For simple, stateless custom operations, you are probably better off using layers.core.Lambda layers. But for any custom operation that has trainable weights, you should implement your own layer.

Here is the skeleton of a Keras layer, as of Keras 2.0 (if you have an older version, please upgrade). There are only three methods you need to implement:

* build(input_shape): this is where you will define your weights. This method must set self.built = True, which can be done by calling super([Layer], self).build().
* call(x): this is where the layer's logic lives. Unless you want your layer to support masking, you only have to care about the first argument passed to call: the input tensor.
* compute_output_shape(input_shape): in case your layer modifies the shape of its input, you should specify here the shape transformation logic. This allows Keras to do automatic shape inference.

In [20]:
from keras.engine.topology import Layer

class MyLayer(Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(shape=(input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        super(MyLayer, self).build(input_shape)  # Be sure to call this somewhere!

    def call(self, x):
        return K.dot(x, self.kernel)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)